### Demo for reading peaks and high-water marks and comparing with model output

Fields in the .csv files are described here: https://my.usgs.gov/confluence/display/WSN/STN+Data+Dictionary+-+Top+Level
(but this will stop working on Jan 27, 2023, and a Sharepoint site will replace it)

This notebook demonstrates reading the `.csv` files, picking out useful data, saveing those as smaller `.csv` files, plotting the data, and finding the data closest to a target lat/lon.

csherwood@usgs.gov

In [ ]:
import pandas as pd
import numpy as np

import xroms
import matplotlib.pyplot as plt
from scipy import stats

# These files were downloaded from https://stn.wim.usgs.gov/fev/#2022Ian on 1/17/2023
hwm_file = r'/vortexfs1/home/csherwood/proj/NOPP/data/FilteredHWMs.csv'
peaks_file = r'/vortexfs1/home/csherwood/proj/NOPP/data/FilteredPeaks.csv'
inst_file = r'/vortexfs1/home/csherwood/proj/NOPP/data/FilteredInstrument.csv'

### Load the peaks data.
Examination reveals:  
* No time or stage estimates were made
* vdatum = NAVD88 except for one point, which we drop
* height_above_gnd is all NaNs
* all times are UTC

In [ ]:
df = pd.read_csv(peaks_file)
#drop the record with local coordinates
df = df.drop(labels=3, axis=0)
# make a copy of the dataframe with the stuff we want in it.
df_peaks = df[['latitude_dd','longitude_dd','peak_date']].copy()
df_peaks['peak_stage_m'] = df['peak_stage'].values*0.3048
df_peaks.describe()

### Read the high-water mark file
Examination reveals:  
    * All vdatum are NAVD88

Values for `hwm_type_id`
```
HWM type code.  Current possibilities:
1 Mud
2 Debris
3 Clear water
4 Vegetation line
5 Seed line
6 Stain line
7 Melted snow line
8 Present at peak (direct observation)
9 Other (Note in Description box)
```
Values for `hwm_quality_id`
```
HWM quality code.  Current possibilities:
1 Excellent: +/- 0.05 ft
2 Good: +/- 0.10 ft
3 Fair: +/- 0.20 ft
4 Poor: +/- 0.40 ft
5 VP: > 0.40 ft
6 Unknown/Historical
```
`stillwater` is boolean
    


In [ ]:
df1 = pd.read_csv(hwm_file)
df1

### Pull a subset of data from the dataframe. 
All of the data for H. Ian are in NAVD88, so not testing for that.

In [ ]:
# Pull data we want into a dataset, rename as 'df_hwm'
df_hwm = df1[['latitude_dd','longitude_dd','hwm_type_id','hwm_quality_id','stillwater']].copy()

# add some columns with data converted to meters
df_hwm['elev_m'] = df1['elev_ft'].values * 0.3048
df_hwm['uncertainty_m'] = df1['uncertainty'].values * 0.3048
df_hwm['height_above_gnd_m'] = df1['height_above_gnd'].values * 0.3048

# you could save this as a .csv file using:
df_hwm.to_csv('hwm_data.csv', index=False, float_format = '%.6f')

# get the statistics for each column
# note: not all of the columns have data...some are missing uncertainty and/or height_above_ground
df_hwm.describe()

In [ ]:
# For fun, make a map of the height above ground of the high-water marks
df_hwm.plot.scatter(x='longitude_dd', y='latitude_dd', s = 25, c = 'height_above_gnd_m' )

#### Demonstrate finding result closest to a target lat/long (not actually needed)

In [ ]:
# Calc. distances from target with Haversine formula
# Modified after https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas

# Provide target lat/lon...the formula requires radians
tlat = np.deg2rad( 27.8 )
tlon = np.deg2rad( -81.7 )
df_hwm[["lat_rad", "lon_rad"]] = np.deg2rad(df_hwm[['latitude_dd', 'longitude_dd']])
dlon = df_hwm['lon_rad'].values - tlon
dlat = df_hwm['lat_rad'].values - tlat

a = np.sin(dlat/2.0)**2 + np.cos(tlat) * np.cos(df_hwm['lat_rad'].values) * np.sin(dlon/2.0)**2
# multiply by radius of earth to get result in km
dist = 6367.* 2. * np.arcsin(np.sqrt(a))

# index of closest one
imin = np.argmin( dist )

print(dist[imin])

print(df_hwm.iloc[imin])

#### Load the model output

In [ ]:
url='http://geoport.whoi.edu/thredds/dodsC/vortexfs1/usgs/Projects/Ian2022/ian5/qck/ian_ocean_sanibel_qck.nc'
ds=xroms.open_netcdf(url)

# define bounding box (This is too big)
lonmin = -82.7
lonmax = -80.3
latmin = 24.
latmax = 31.

# this condition defines the region of interest
box = ((lonmin < ds.lon_rho) & (ds.lon_rho < lonmax) & (latmin < ds.lat_rho) & (ds.lat_rho < latmax)).compute()

In [ ]:
ds

In [ ]:
dss=ds.where(box, drop=True).zeta
dss

In [ ]:
# Get wet/dry mask (1=wet, 0=dry) and calculate max (should correspond with max water levels)
dswd = ds.where(box, drop=True).wetdry_mask_rho
wd = dswd.max(dim='ocean_time').compute()

In [ ]:
wd.plot(x="lon_rho", y="lat_rho", cmap='coolwarm')

In [ ]:
# find the maximum water level over the entire time
# adding the .compute() function was key to speeding up the lat/lon lookup below
zmax = dss.max(dim='ocean_time').compute()

In [ ]:
# TODO - make real maps
vmin=0
vmax=4
fig = plt.figure()
zmax.plot(x="lon_rho", y="lat_rho", vmin=vmin, vmax=vmax, cmap='coolwarm')
# Make a map of the height above ground of the high-water marks
plt.scatter( df_hwm['longitude_dd'], df_hwm['latitude_dd'], s=30, c = df_hwm['elev_m'], \
            edgecolor='dimgray',vmin=vmin, vmax=vmax, label='HW Marks',cmap='coolwarm' )
plt.scatter( df_peaks['longitude_dd'], df_peaks['latitude_dd'], s=30, c = df_peaks['peak_stage_m'], \
            edgecolor='dimgray', vmin=vmin, vmax=vmax, marker = '*', label='Peaks',cmap='coolwarm' )
plt.xlim([-82.6,-81.5])
plt.ylim([26.2,27.2])
plt.legend()

In [ ]:
# Loop through the high-water marks and find zmax in nearest model grid point
# This was glacially slow before I added the .compute() function while calculating zmax
meas_hwm = df_hwm['elev_m'].values
meas_qc = df_hwm['hwm_quality_id'].values
model_zeta_hwm = np.nan*np.ones_like( meas_hwm )
hwm_mask = np.nan*np.ones_like( meas_hwm )

for i, (lonr, latr) in enumerate( zip(df_hwm['longitude_dd'].values, df_hwm['latitude_dd'].values) ):
    model_zeta_hwm[i] =  zmax.xroms.sel2d(lonr, latr).values
    hwm_mask[i] = wd.xroms.sel2d(lonr, latr).values

meas_peaks = df_peaks['peak_stage_m'].values
model_zeta_peaks = np.nan*np.ones_like( meas_peaks )
peak_mask = np.nan*np.ones_like( meas_peaks )
for i, (lonr, latr) in enumerate( zip(df_peaks['longitude_dd'].values, df_peaks['latitude_dd'].values) ):
    model_zeta_peaks[i] =  zmax.xroms.sel2d(lonr, latr).values
    peak_mask[i] =  wd.xroms.sel2d(lonr, latr).values

In [ ]:
mod_hwm_good = model_zeta_hwm[meas_qc<3][hwm_mask[meas_qc<3]==1]
meas_hwm_good = meas_hwm[meas_qc<3][hwm_mask[meas_qc<3]==1]
mod_hwm_other = model_zeta_hwm[meas_qc>=3][hwm_mask[meas_qc>=3]==1]
meas_hwm_other = meas_hwm[meas_qc>=3][hwm_mask[meas_qc>=3]==1]
model_peaks = model_zeta_peaks[peak_mask==1]
meas_mpeaks = meas_peaks[peak_mask==1]

gradient_hwm_good, intercept_hwm_good, r_value, p_value, std_err = stats.linregress(mod_hwm_good, meas_hwm_good)
ts_good = 'HWM, Good QC: $r^2$={:.2f}, slope={:.2f}'.format(r_value,gradient_hwm_good)
print(ts_good)
gradient_hwm_other, intercept_hwm_other, r_value, p_value, std_err = stats.linregress(mod_hwm_other, meas_hwm_other)
ts_other = 'HWM, Other QC: $r^2$={:.2f}, slope={:.2f}'.format(r_value,gradient_hwm_fair)
print(ts_other)
gradient_peaks, intercept_peaks, r_value, p_value, std_err = stats.linregress(model_peaks, meas_mpeaks)
ts_peaks = 'Peaks: $r^2$={:.2f}, slope={:.2f}'.format(r_value,gradient_peaks)
print(ts_peaks)
x_good = np.array((np.min(mod_hwm_good), np.max(mod_hwm_good)))
y_good = intercept_hwm_good + gradient_hwm_good * x_good
x_other = np.array((np.min(mod_hwm_other), np.max(mod_hwm_other)))
y_other = intercept_hwm_other + gradient_hwm_other * x_other
x_peaks = np.array((np.min(model_peaks), np.max(model_peaks)))
y_peaks = intercept_peaks + gradient_peaks * x_peaks

In [ ]:
cols = ['#e66101','#fdb863','#b2abd2','#5e3c99']

fig, ax = plt.subplots(1, 1, figsize=(8, 6))  # setup the plot
plt.plot([0,4.5],[0,4.5],'--',c='grey')
plt.plot( x_good, y_good, '--',c=cols[3])
plt.plot( x_other, y_other, '--',c=cols[2])
plt.plot( x_peaks, y_peaks, '--',c=cols[0])
plt.plot(mod_hwm_other, meas_hwm_other,'o',c=cols[2],label='HWM (QC > 2)')
plt.plot(mod_hwm_good, meas_hwm_good,'o',c=cols[3],label='HWM (QC <= 2')
plt.plot(model_zeta_peaks[peak_mask==1], meas_peaks[peak_mask==1],'s', c=cols[0], label='Peaks')

plt.ylabel('Observed Max. Water Levels (m NAVD88)')
plt.xlabel('Modeled Max. Water Levels (m MSL)')
plt.xlim([0,4.5])
plt.ylim([0,4.5])

# chagne the order of stuff in the legend
#get handles and labels
handles, labels = plt.gca().get_legend_handles_labels()
#specify order of items in legend
order = [2,1,0]
#add legend to plot
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order])

#plt.legend()
plt.text(x=1.05, y=.90, s=ts_peaks, fontsize=10, c='k',transform=ax.transAxes,\
             horizontalalignment='left', verticalalignment='bottom')
plt.text(x=1.05, y=.85, s=ts_good, fontsize=10, c='k',transform=ax.transAxes,\
             horizontalalignment='left', verticalalignment='bottom')
plt.text(x=1.05, y=.80, s=ts_other, fontsize=10, c='k',transform=ax.transAxes,\
             horizontalalignment='left', verticalalignment='bottom')



ax.set_aspect(1)
plt.tight_layout()

plt.savefig('hwm_v_model.png',dpi=400)
plt.savefig('hwm_v_model.svg',dpi=400)